## Идеи

1. Необходимо классифицировать изображения на 12 классов. Используем простую сверточную сеть с двумя полносвязными слоями. На выходе - слой размера 12 с softmax. На входе - тензор батча изобржаний
2. Изображения очень похожи друг на друга. В центре - побег растения, фон - почва (камни, замля и т.д.)
3. Изображения не стандартного формата (вроде). Необходимо привести к одному масштабу. 
4. Так как фото сделаны сверху - нет строгой ориентации побега (однако, он центрирован) и нет строгого отношения размер изображения/размер побега. Поэтому точно классной идеей будет применить аугментацию изображений поворотом и масштабированием. Возможно, также сдвигом. ImageDataGenerator
5. В качестве доп. фичей можно применить гистограмму цветов в области побега
6. Обучить несколько независимых сетей на частях данных (допустим, 4), делать стекинг на тестовой выборке из их предсказаний. Сам стекинг не обучать (для скорости)
7. Попробовать дообучать несколько слоев ResNet (хотя, эта сеть обучена для слишком большого количества изображений)
8. После обучения смотреть матрицу ошибок, выделять дополнительно то, что можно дообучить в случае больших ошибок сети
9. Подумать над размером и стратификацией батча (видел батчи по 16-20 фото, но у нас 12 классов. Так, есть большая вероятность не попадания нескольких классов в батч)

## imports

In [1]:
import pandas as pd
import numpy as np
import os
import imageio

from keras.utils import plot_model
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm
import os
import numpy as np
import cv2
from glob import glob
from sklearn.decomposition import PCA

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# global vars

In [24]:
BATCH_SIZE = 16
EPOCHS = 10
RANDOM_STATE = 5
CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

# Создание сети

In [5]:
def dense_set(inp_layer, n, activation, drop_rate=0.):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act

In [6]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act

In [21]:
def get_model():
    inp_img = Input(shape=(51, 51, 3))

    # 51
    conv1 = conv_layer(inp_img, 16, zp_flag=False)
    conv2 = conv_layer(conv1, 16, zp_flag=False)
    mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
    # 23
    conv3 = conv_layer(mp1, 32, zp_flag=False)
    conv4 = conv_layer(conv3, 32, zp_flag=False)
    mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
    # 9
    conv7 = conv_layer(mp2, 64, zp_flag=False)
    conv8 = conv_layer(conv7, 64, zp_flag=False)
    conv9 = conv_layer(conv8, 64, zp_flag=False)
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)
    # 1
    # dense layers
    flt = Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = Model(inputs=inp_img, outputs=out)

    
    mypotim = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#     mypotim = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                   optimizer=mypotim,
                   metrics=['accuracy'])
    model.summary()
    return model

In [8]:
def get_callbacks(filepath, patience=5):
    lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=1e-5, patience=patience, verbose=1)
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [lr_reduce, msave]

In [9]:
# Архитектура MNIST, не применяю тут
def gen_model():
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(51, 51, 3)),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(rate=0.25),
        Flatten(),
        Dense(1024, activation='relu'),
        Dense(12, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Методы для обучения и предсказания

In [10]:
def train_model(img, target):
    callbacks = get_callbacks(filepath='model_weight_Adam.hdf5', patience=6)
    gmodel = get_model()
#     gmodel.load_weights(filepath='model_weight_Adam.hdf5')
    x_train, x_valid, y_train, y_valid = train_test_split(
                                                        img,
                                                        target,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=RANDOM_STATE
                                                        )
    gen = ImageDataGenerator(
            rotation_range=180.,
            width_shift_range=0.4,
            height_shift_range=0.4,
            zoom_range=0.5,
            vertical_flip=True
    )
    gmodel.fit_generator(gen.flow(x_train, y_train,batch_size=BATCH_SIZE),
               steps_per_epoch=len(x_train)/BATCH_SIZE,
               epochs=EPOCHS,
               verbose=1,
               shuffle=True,
               validation_data=(x_valid, y_valid),
               callbacks=callbacks
               )
#     

In [26]:
def test_model(img, label):
    gmodel = get_model()
    gmodel.load_weights(filepath='model_weight_Adam.hdf5')
    prob = gmodel.predict(img, verbose=1)
    pred = prob.argmax(axis=-1)
    sub = pd.DataFrame({"file": label,
                         "species": [INV_CLASS[p] for p in pred]})
    sub.to_csv("sub.csv", index=False, header=True)

# чтение и предобработка данных

In [13]:
def img_reshape(img):
    img = imresize(img, (51, 51, 3))
    return img

In [14]:
def img_label(path):
    return str(str(path.split('\\')[-1]))

In [15]:
def img_class(path):
    return str(path.split('\\')[-2])

In [16]:
def fill_dict(paths, some_dict):
    text = ''
    if 'train' in paths[0]:
        text = 'Start fill train_dict'
    elif 'test' in paths[0]:
        text = 'Start fill test_dict'

    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = cv2.imread(p)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)    
        mask = cv2.inRange(hsv, (24, 50, 0), (55, 255, 140))
        img = cv2.bitwise_and(img, img, mask=mask)
        img = img_reshape(img)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))
        if 'train' in paths[0]:
            some_dict['class'].append(img_class(p))
    return some_dict

In [17]:
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('Data-hackaton'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]

                if ext not in file_ext:
                    file_ext.append(ext)

                if 'train' in root:
                    path = os.path.join(root, f)
                    train_path.append(path)
                elif 'test' in root:
                    path = os.path.join(root, f)
                    test_path.append(path)
    train_dict = {
        'image': [],
        'label': [],
        'class': []
    }
    test_dict = {
        'image': [],
        'label': []
    }

    train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

## Собственно, чтение данных в словарь

In [18]:
train_dict, test_dict = reader()

Root:
Data-hackaton
Dirs:
['test', 'train']
Root:
Data-hackaton\train
Dirs:
['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


Start fill train_dict:   0%|                                | 0/4750 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Start fill test_dict: 100%|########################| 794/794 [00:27<00:00, 28.96it/s]


## Составление необходимых ndarray, передаваемых в модель

In [22]:
X_train = np.array(train_dict['image'])
y_train = to_categorical(np.array([CLASS[l] for l in train_dict['class']]))

In [23]:
X_test = np.array(test_dict['image'])
label = test_dict['label']

## Обучение модели

In [25]:
train_model(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 51, 51, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 49, 49, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 49, 16)        64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 47, 47, 16)        64        
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 47, 47, 16)        0         
__________

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Epoch 1/10


206/237 [=========================>....] - ETA: 21:28 - loss: 3.0258 - acc: 0.0000e+ - ETA: 12:07 - loss: 2.8841 - acc: 0.0312   - ETA: 8:59 - loss: 2.7663 - acc: 0.1042 - ETA: 7:25 - loss: 2.7226 - acc: 0.109 - ETA: 6:28 - loss: 2.7058 - acc: 0.112 - ETA: 5:50 - loss: 2.7540 - acc: 0.104 - ETA: 5:23 - loss: 2.7088 - acc: 0.116 - ETA: 5:02 - loss: 2.7494 - acc: 0.117 - ETA: 4:46 - loss: 2.6952 - acc: 0.131 - ETA: 4:32 - loss: 2.6778 - acc: 0.143 - ETA: 4:21 - loss: 2.6897 - acc: 0.153 - ETA: 4:12 - loss: 2.6722 - acc: 0.161 - ETA: 4:05 - loss: 2.7063 - acc: 0.149 - ETA: 3:58 - loss: 2.6849 - acc: 0.147 - ETA: 3:52 - loss: 2.6599 - acc: 0.158 - ETA: 3:46 - loss: 2.6659 - acc: 0.152 - ETA: 3:41 - loss: 2.6492 - acc: 0.158 - ETA: 3:37 - loss: 2.6623 - acc: 0.159 - ETA: 3:33 - loss: 2.6540 - acc: 0.151 - ETA: 3:30 - loss: 2.6299 - acc: 0.156 - ETA: 3:26 - loss: 2.6136 - acc: 0.160 - ETA: 3:23 - loss: 2.6004 - acc: 0.167 - ETA: 3:20 - loss: 2.5990 - acc: 0.171 - ETA: 3:18 - loss: 2.5908 - a

...

238/237 [==============================] - ETA: 22s - loss: 1.0923 - acc: 0.64 - ETA: 21s - loss: 1.0939 - acc: 0.63 - ETA: 20s - loss: 1.0951 - acc: 0.63 - ETA: 19s - loss: 1.0932 - acc: 0.64 - ETA: 19s - loss: 1.0939 - acc: 0.64 - ETA: 18s - loss: 1.0949 - acc: 0.64 - ETA: 17s - loss: 1.0953 - acc: 0.64 - ETA: 16s - loss: 1.0948 - acc: 0.64 - ETA: 16s - loss: 1.0948 - acc: 0.64 - ETA: 15s - loss: 1.0939 - acc: 0.64 - ETA: 14s - loss: 1.0945 - acc: 0.63 - ETA: 13s - loss: 1.0927 - acc: 0.64 - ETA: 13s - loss: 1.0920 - acc: 0.64 - ETA: 12s - loss: 1.0916 - acc: 0.64 - ETA: 11s - loss: 1.0911 - acc: 0.64 - ETA: 10s - loss: 1.0914 - acc: 0.64 - ETA: 10s - loss: 1.0921 - acc: 0.64 - ETA: 9s - loss: 1.0941 - acc: 0.6417 - ETA: 8s - loss: 1.0940 - acc: 0.641 - ETA: 7s - loss: 1.0935 - acc: 0.642 - ETA: 7s - loss: 1.0939 - acc: 0.641 - ETA: 6s - loss: 1.0971 - acc: 0.640 - ETA: 5s - loss: 1.0997 - acc: 0.639 - ETA: 4s - loss: 1.0986 - acc: 0.639 - ETA: 4s - loss: 1.0993 - acc: 0.639 - ETA: 3

207/237 [=========================>....] - ETA: 2:49 - loss: 0.8968 - acc: 0.687 - ETA: 2:51 - loss: 0.9066 - acc: 0.718 - ETA: 2:50 - loss: 1.0130 - acc: 0.666 - ETA: 2:50 - loss: 1.0526 - acc: 0.671 - ETA: 2:50 - loss: 1.0813 - acc: 0.662 - ETA: 2:49 - loss: 1.0339 - acc: 0.677 - ETA: 2:48 - loss: 0.9712 - acc: 0.696 - ETA: 2:48 - loss: 0.9893 - acc: 0.687 - ETA: 2:48 - loss: 1.0440 - acc: 0.666 - ETA: 2:46 - loss: 1.0245 - acc: 0.675 - ETA: 2:46 - loss: 1.0015 - acc: 0.676 - ETA: 2:46 - loss: 1.0050 - acc: 0.671 - ETA: 2:46 - loss: 0.9966 - acc: 0.673 - ETA: 2:48 - loss: 0.9973 - acc: 0.669 - ETA: 2:48 - loss: 1.0109 - acc: 0.666 - ETA: 2:48 - loss: 1.0091 - acc: 0.671 - ETA: 2:48 - loss: 1.0098 - acc: 0.669 - ETA: 2:50 - loss: 1.0068 - acc: 0.673 - ETA: 2:50 - loss: 0.9933 - acc: 0.684 - ETA: 2:49 - loss: 0.9714 - acc: 0.696 - ETA: 2:48 - loss: 0.9639 - acc: 0.702 - ETA: 2:47 - loss: 0.9624 - acc: 0.698 - ETA: 2:46 - loss: 0.9760 - acc: 0.692 - ETA: 2:45 - loss: 0.9736 - acc: 0.690

238/237 [==============================] - ETA: 22s - loss: 1.0432 - acc: 0.66 - ETA: 21s - loss: 1.0434 - acc: 0.66 - ETA: 20s - loss: 1.0427 - acc: 0.66 - ETA: 19s - loss: 1.0417 - acc: 0.66 - ETA: 19s - loss: 1.0404 - acc: 0.66 - ETA: 18s - loss: 1.0398 - acc: 0.66 - ETA: 17s - loss: 1.0410 - acc: 0.66 - ETA: 16s - loss: 1.0414 - acc: 0.66 - ETA: 16s - loss: 1.0423 - acc: 0.66 - ETA: 15s - loss: 1.0413 - acc: 0.66 - ETA: 14s - loss: 1.0414 - acc: 0.66 - ETA: 13s - loss: 1.0404 - acc: 0.66 - ETA: 13s - loss: 1.0404 - acc: 0.66 - ETA: 12s - loss: 1.0399 - acc: 0.66 - ETA: 11s - loss: 1.0414 - acc: 0.66 - ETA: 10s - loss: 1.0409 - acc: 0.66 - ETA: 10s - loss: 1.0406 - acc: 0.66 - ETA: 9s - loss: 1.0420 - acc: 0.6625 - ETA: 8s - loss: 1.0428 - acc: 0.661 - ETA: 7s - loss: 1.0414 - acc: 0.662 - ETA: 7s - loss: 1.0403 - acc: 0.663 - ETA: 6s - loss: 1.0417 - acc: 0.662 - ETA: 5s - loss: 1.0407 - acc: 0.663 - ETA: 4s - loss: 1.0392 - acc: 0.663 - ETA: 4s - loss: 1.0389 - acc: 0.663 - ETA: 3

207/237 [=========================>....] - ETA: 2:53 - loss: 1.0278 - acc: 0.687 - ETA: 2:53 - loss: 1.0161 - acc: 0.625 - ETA: 2:51 - loss: 0.9713 - acc: 0.666 - ETA: 2:51 - loss: 0.9073 - acc: 0.687 - ETA: 2:50 - loss: 0.9958 - acc: 0.662 - ETA: 2:50 - loss: 0.9814 - acc: 0.666 - ETA: 2:49 - loss: 0.9406 - acc: 0.678 - ETA: 2:49 - loss: 0.9821 - acc: 0.648 - ETA: 2:48 - loss: 0.9589 - acc: 0.666 - ETA: 2:47 - loss: 0.9437 - acc: 0.675 - ETA: 2:47 - loss: 0.9504 - acc: 0.681 - ETA: 2:46 - loss: 0.9518 - acc: 0.677 - ETA: 2:45 - loss: 1.0005 - acc: 0.663 - ETA: 2:45 - loss: 0.9909 - acc: 0.674 - ETA: 2:44 - loss: 1.0121 - acc: 0.658 - ETA: 2:44 - loss: 1.0146 - acc: 0.656 - ETA: 2:43 - loss: 1.0031 - acc: 0.658 - ETA: 2:42 - loss: 0.9851 - acc: 0.670 - ETA: 2:41 - loss: 0.9875 - acc: 0.671 - ETA: 2:40 - loss: 0.9874 - acc: 0.671 - ETA: 2:40 - loss: 1.0001 - acc: 0.663 - ETA: 2:39 - loss: 0.9947 - acc: 0.667 - ETA: 2:38 - loss: 0.9837 - acc: 0.676 - ETA: 2:37 - loss: 0.9635 - acc: 0.684

238/237 [==============================] - ETA: 22s - loss: 1.0480 - acc: 0.65 - ETA: 22s - loss: 1.0482 - acc: 0.65 - ETA: 21s - loss: 1.0491 - acc: 0.65 - ETA: 20s - loss: 1.0491 - acc: 0.65 - ETA: 19s - loss: 1.0464 - acc: 0.65 - ETA: 19s - loss: 1.0460 - acc: 0.65 - ETA: 18s - loss: 1.0454 - acc: 0.65 - ETA: 17s - loss: 1.0459 - acc: 0.65 - ETA: 16s - loss: 1.0445 - acc: 0.65 - ETA: 15s - loss: 1.0454 - acc: 0.65 - ETA: 15s - loss: 1.0443 - acc: 0.65 - ETA: 14s - loss: 1.0466 - acc: 0.65 - ETA: 13s - loss: 1.0496 - acc: 0.65 - ETA: 12s - loss: 1.0505 - acc: 0.65 - ETA: 12s - loss: 1.0497 - acc: 0.65 - ETA: 11s - loss: 1.0510 - acc: 0.65 - ETA: 10s - loss: 1.0509 - acc: 0.65 - ETA: 9s - loss: 1.0507 - acc: 0.6556 - ETA: 8s - loss: 1.0511 - acc: 0.655 - ETA: 8s - loss: 1.0513 - acc: 0.656 - ETA: 7s - loss: 1.0528 - acc: 0.655 - ETA: 6s - loss: 1.0525 - acc: 0.655 - ETA: 5s - loss: 1.0535 - acc: 0.655 - ETA: 5s - loss: 1.0522 - acc: 0.656 - ETA: 4s - loss: 1.0518 - acc: 0.656 - ETA: 3

In [27]:
test_model(X_test, label)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 51, 51, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 49, 49, 16)        448       
_________________________________________________________________
batch_normalization_10 (Batc (None, 49, 49, 16)        64        
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 47, 47, 16)        2320      
_________________________________________________________________
batch_normalization_11 (Batc (None, 47, 47, 16)        64        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 47, 47, 16)        0         
__________

In [28]:
# 0.6801